In [1]:
#!pip install seaborn 

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
sns.__version__

'0.11.2'

In [5]:
FOLDER = 'data'
FILE = '20-years-odds.csv'


data = pd.read_csv(f'{FOLDER}/{FILE}')
LIMIT = int(data.shape[0]*0.85)

data.shape, LIMIT

((3310, 17), 2813)

In [6]:
data.head()

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,Arsenal,Everton,5:1,1.34,5.50,8.98,2022-05-22,2022-05,england,2022,premier-league-2021,5.0,1.0,1,0.0,1.34,True
1,Brentford,Leeds,1:2,2.32,3.79,2.88,2022-05-22,2022-05,england,2022,premier-league-2021,1.0,2.0,2,2.0,2.88,False
2,Brighton,West Ham,3:1,2.66,3.46,2.65,2022-05-22,2022-05,england,2022,premier-league-2021,3.0,1.0,1,0.0,2.66,False
3,Burnley,Newcastle,1:2,2.47,3.27,3.01,2022-05-22,2022-05,england,2022,premier-league-2021,1.0,2.0,2,2.0,3.01,False
4,Chelsea,Watford,2:1,1.19,7.41,15.39,2022-05-22,2022-05,england,2022,premier-league-2021,2.0,1.0,1,0.0,1.19,True


In [7]:
teams = [data.home_team.tolist()] + [data.away_team.tolist()]

teams = np.unique(teams[0])
len(np.unique(teams))

166

In [8]:
scores = np.unique(data.correct_score)
df_scores = pd.DataFrame(data.correct_score.value_counts(normalize=True))
#df_scores.columns = [['scores','ratio']]
#df_scores.sort_values(ascending=False)
df_scores.columns = ['ratio']
df_scores.sort_values('ratio',ascending=False)

,ratio
1:1,0.131420
1:0,0.101813
2:1,0.083082
2:0,0.079456
0:1,0.076737
1:2,0.072810
0:0,0.069486
2:2,0.049547
0:2,0.048943
3:1,0.035347


# Baseline

Sinmply compute a baseline by taking the 1N2 odds, transpose them to 1/odds then scale date and train a multi-output regressor.

In [9]:
df = data[['1N2_1','1N2_N','1N2_2','home_score','away_score']]
df

,1N2_1,1N2_N,1N2_2,home_score,away_score
0,1.34,5.50,8.98,5.0,1.0
1,2.32,3.79,2.88,1.0,2.0
2,2.66,3.46,2.65,3.0,1.0
3,2.47,3.27,3.01,1.0,2.0
4,1.19,7.41,15.39,2.0,1.0
...,...,...,...,...,...
3305,6.85,4.03,1.53,2.0,2.0
3306,4.04,3.75,1.88,1.0,3.0
3307,1.29,5.53,10.81,4.0,0.0
3308,6.12,4.46,1.52,1.0,2.0


In [36]:
X = 1/df[['1N2_1','1N2_N','1N2_2']]
y = df.drop(['1N2_1','1N2_N','1N2_2'],axis=1)
# summarize dataset
print(X.shape, y.shape)

(3310, 3) (3310, 2)


In [13]:
X_train=X[:LIMIT]
X_test=X[LIMIT:]

y_train = y[:LIMIT]
y_test = y[LIMIT:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2813, 3) (497, 3) (2813, 2) (497, 2)


In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
model = LinearRegression(fit_intercept=False)

model = DecisionTreeRegressor()

model = KNeighborsRegressor(n_neighbors=18)

model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=18)

In [55]:

yhat = model.predict(X_test)
eval = y_test.copy()

eval['home_pred']=yhat[:,0]
eval['away_pred']=yhat[:,1]

conditions = [
    ((eval['home_pred'] > eval['away_pred']) & (eval['home_score'] > eval['away_score'])),
    ((eval['home_pred'] == eval['away_pred']) & (eval['home_score'] == eval['away_score'])),
    ((eval['home_pred'] < eval['away_pred']) & (eval['home_score'] < eval['away_score']))
    
]
choices = [True,True,True]

eval['prediction'] = np.select(conditions, choices, default=False)
eval.prediction.value_counts(normalize=True)

False    0.519115
True     0.480885
Name: prediction, dtype: float64

In [45]:
eval

,home_score,away_score,home_pred,away_pred,prediction
2813,0.0,2.0,1.105263,1.526316,True
2814,2.0,1.0,1.631579,0.736842,True
2815,1.0,4.0,1.368421,1.210526,False
2816,1.0,0.0,1.894737,0.736842,True
2817,4.0,1.0,1.157895,1.000000,True
...,...,...,...,...,...
3305,2.0,2.0,0.684211,1.526316,False
3306,1.0,3.0,1.210526,1.736842,True
3307,4.0,0.0,2.368421,0.578947,True
3308,1.0,2.0,0.894737,2.105263,True


In [47]:
eval.query('prediction == False')

,home_score,away_score,home_pred,away_pred,prediction
2815,1.0,4.0,1.368421,1.210526,False
2818,1.0,1.0,1.631579,1.315789,False
2820,0.0,1.0,1.789474,1.157895,False
2825,1.0,1.0,1.684211,1.263158,False
2826,2.0,2.0,1.000000,0.947368,False
...,...,...,...,...,...
3296,0.0,1.0,2.789474,0.789474,False
3300,2.0,2.0,1.947368,0.894737,False
3301,0.0,1.0,1.157895,0.947368,False
3305,2.0,2.0,0.684211,1.526316,False
